In [3]:
# Run in python console
import nltk; nltk.download('stopwords')

# Run in terminal or command prompt
#python3 -m spacy download en

[nltk_data] Downloading package stopwords to /home/levin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import matplotlib.pyplot as plt


# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

/home/levin/Python/arabpdf/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [5]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [6]:
# Import Dataset
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df.target_names.unique())
df.head()

['rec.autos' 'comp.sys.mac.hardware' 'comp.graphics' 'sci.space'
 'talk.politics.guns' 'sci.med' 'comp.sys.ibm.pc.hardware'
 'comp.os.ms-windows.misc' 'rec.motorcycles' 'talk.religion.misc'
 'misc.forsale' 'alt.atheism' 'sci.electronics' 'comp.windows.x'
 'rec.sport.hockey' 'rec.sport.baseball' 'soc.religion.christian'
 'talk.politics.mideast' 'talk.politics.misc' 'sci.crypt']


,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


In [7]:
# Convert to list
data = df.content.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']


In [8]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


In [9]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd_edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


In [10]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [11]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load("en_core_web_sm")

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['where', 's', 'thing', 'car', 'nntp_poste', 'host', 'rac_wam', 'umd', 'organization', 'university', 'park', 'line', 'wonder', 'enlighten', 'car', 'saw', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'bricklin', 'door', 'really', 'small', 'addition', 'separate', 'rest', 'body', 'know', 'tellme', 'model', 'name', 'engine', 'spec', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


### Try Inverting Words

In [12]:
data_words_nostops_inverted = []
for i in data_lemmatized:
    temp = [j[::-1] for j in i]
    data_words_nostops_inverted.append(temp)

### Continue gensim procedure

In [13]:
# Create Dictionary
id2word = corpora.Dictionary(data_words_nostops_inverted)

# Create Corpus
texts = data_words_nostops_inverted

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 2), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 5), (29, 1), (30, 1), (31, 2), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1)]]


In [14]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('ceps', 1),
  ('dmu', 1),
  ('doohrobhgien', 1),
  ('ekam', 1),
  ('eman', 1),
  ('emllet', 1),
  ('enigne', 1),
  ('enil', 1),
  ('erehw', 1),
  ('etal', 1),
  ('etarapes', 1),
  ('etsop_ptnn', 1),
  ('gniht', 1),
  ('gnirb', 1),
  ('knaht', 1),
  ('kool', 2),
  ('krap', 1),
  ('ledom', 1),
  ('liam', 1),
  ('llac', 1),
  ('llams', 1),
  ('maw_car', 1),
  ('nethgilne', 1),
  ('nilkcirb', 1),
  ('noitazinagro', 1),
  ('noitcudorp', 1),
  ('noitidda', 1),
  ('ofni', 1),
  ('rac', 5),
  ('raey', 1),
  ('rednow', 1),
  ('rood', 2),
  ('s', 1),
  ('trops', 1),
  ('tser', 1),
  ('tsoh', 1),
  ('tsxrel', 1),
  ('was', 1),
  ('wonk', 1),
  ('yad', 1),
  ('ydob', 1),
  ('yknuf', 1),
  ('yllaer', 1),
  ('ylrae', 1),
  ('yrotsih', 1),
  ('ytisrevinu', 1)]]

In [15]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [16]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.075*"evird" + 0.062*"wodniw" + 0.046*"drac" + 0.035*"cam" + '
  '0.033*"revird" + 0.026*"upc" + 0.024*"melborp" + 0.023*"yromem" + '
  '0.021*"cihparg" + 0.021*"neercs"'),
 (1,
  '0.060*"evets" + 0.054*"weiver" + 0.030*"enalp" + 0.024*"naidanac" + '
  '0.023*"retaw" + 0.022*"pam" + 0.020*"raelcun" + 0.020*"ecnerefnoc" + '
  '0.020*"noitartsiger" + 0.019*"ailartsua"'),
 (2,
  '0.097*"enil" + 0.087*"noitazinagro" + 0.051*"etsop_ptnn" + '
  '0.048*"ytisrevinu" + 0.046*"tsoh" + 0.026*"ylper" + 0.024*"knaht" + '
  '0.020*"liam" + 0.016*"tsop" + 0.013*"pleh"'),
 (3,
  '0.089*"ap" + 0.058*"retnirp" + 0.050*"sm" + 0.033*"yob" + 0.031*"aissur" + '
  '0.018*"noitcerroc" + 0.016*"tluc" + 0.010*"rev" + 0.006*"ynomrah" + '
  '0.005*"enegue"'),
 (4,
  '0.033*"llik" + 0.030*"nainemra" + 0.027*"raw" + 0.025*"reidlos" + '
  '0.023*"keerg" + 0.022*"egalliv" + 0.019*"redrum" + 0.019*"krut" + '
  '0.018*"htaed" + 0.017*"milsum"'),
 (5,
  '0.173*"nug" + 0.053*"mraerif" + 0.049*"nopaew" + 0.026*"e

In [17]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words_nostops_inverted, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -14.364487207780734

Coherence Score:  0.5182625811151266


In [18]:
#inserted import (old version conflict solved)
import pyLDAvis.gensim_models

# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9     -0.359984  0.058513       1        1  41.624886
12    -0.252661 -0.148240       2        1   9.766968
2     -0.235236 -0.170215       3        1   9.288860
6     -0.301914  0.177762       4        1   8.730195
10    -0.170629  0.194753       5        1   5.948657
18     0.022621 -0.036901       6        1   5.544736
17    -0.064420 -0.150838       7        1   3.584368
0     -0.020962 -0.220133       8        1   2.604725
15     0.002082 -0.001346       9        1   2.598467
13    -0.009419  0.048288      10        1   2.483097
4      0.009445  0.170727      11        1   1.908615
19     0.115882 -0.081703      12        1   0.947664
7      0.126912  0.031188      13        1   0.910990
16     0.140805  0.018171      14        1   0.872379
14     0.150740  0.013390      15        1   0.851275
8      0.126356  0.011966      16        1   0.814512
1      0.168583  0.018114      17        1   0.551949
5      0.175413  0.026874      18        1   0.438776
11     0.181087  0.018998      19        1   0.346873
3      0.195299  0.020632      20        1   0.182008, topic_info=               Term          Freq         Total Category  logprob  loglift
5686             xa  52570.000000  52570.000000  Default  30.0000   30.000
7              enil  18952.000000  18952.000000  Default  29.0000   29.000
24     noitazinagro  17400.000000  17400.000000  Default  28.0000   28.000
1080              _   8691.000000   8691.000000  Default  27.0000   27.000
11       etsop_ptnn   7224.000000   7224.000000  Default  26.0000   26.000
...             ...           ...           ...      ...      ...      ...
24604           q_r      0.026439      1.311192  Topic20 -11.5534    2.405
5706             xc      0.026443      1.311366  Topic20 -11.5533    2.405
5542             km      0.026440      1.311214  Topic20 -11.5534    2.405
8133             dl      0.026438      1.311205  Topic20 -11.5535    2.405
70047           wiu      0.026437      1.311184  Topic20 -11.5535    2.405

[866 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
1080      6  0.999923             _
3242     13  0.984106    adnagaporp
1773     13  0.987194      aibmuloc
3947     17  0.990897     ailartsua
3949     19  0.984287  ainavlysnnep
...     ...       ...           ...
542       1  0.669729           yub
542       7  0.047879           yub
542       8  0.136048           yub
542      12  0.145974           yub
4030     19  0.973025         yzzuf

[1126 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 13, 3, 7, 11, 19, 18, 1, 16, 14, 5, 20, 8, 17, 15, 9, 2, 6, 12, 4])